In [1]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium
# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelea

In [2]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
from tqdm import tqdm
import logging
import traceback
from selenium.common.exceptions import NoSuchElementException

In [3]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Files-Required/client project')

Mounted at /content/drive


In [4]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

In [5]:
def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

log_filename = 'exemption_logs.txt'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler(log_filename)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logging.getLogger().addHandler(file_handler)


In [6]:
gdown.download(id = '1-0PMng9-Ms_uc0f00KuUgEWXi7xS29Ph')

Downloading...
From: https://drive.google.com/uc?id=1-0PMng9-Ms_uc0f00KuUgEWXi7xS29Ph
To: /content/drive/MyDrive/Files-Required/client project/regions.csv
100%|██████████| 4.61k/4.61k [00:00<00:00, 11.9MB/s]


'regions.csv'

In [7]:
regiondf = pd.read_csv('/content/drive/MyDrive/Files-Required/client project/regions.csv')
regiondf

,region_links
0,https://www.homely.com.au/find-suburb-by-regio...
1,https://www.homely.com.au/find-suburb-by-regio...
2,https://www.homely.com.au/find-suburb-by-regio...
3,https://www.homely.com.au/find-suburb-by-regio...
4,https://www.homely.com.au/find-suburb-by-regio...
...,...
57,https://www.homely.com.au/find-suburb-by-regio...
58,https://www.homely.com.au/find-suburb-by-regio...
59,https://www.homely.com.au/find-suburb-by-regio...
60,https://www.homely.com.au/find-suburb-by-regio...


In [ ]:
driver = driver_setup()
driver.set_page_load_timeout(60)

suburb_links = {}
l = 0

for region_index in range(4):
  region = regiondf['region_links'][region_index]
  driver.get(region)
  suburb_ul_elements = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/main/article/div[2]/div/div[2]/ul')

  for suburb_ul_element in suburb_ul_elements:
        suburb_a_elements = suburb_ul_element.find_elements(By.TAG_NAME, 'a')
        for suburb_a_element in suburb_a_elements:                         # link extractor block
            try:
              href = suburb_a_element.get_attribute('href')
              suburb_links.update({l:[href,region]})
              l+=1
            except:
              None
for index,sub_reg in suburb_links.items():
  suburb = sub_reg[0]
  region = sub_reg[1]

  driver.get(suburb)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')

  try:
    suburbname = soup.find('h1').get_text()
  except:
    suburbname = ''
  try:
    ratings = soup.find('span', class_='sc-1svr2te-6 jRYGqL').get_text()
  except:
    ratings = ''
  try:
    no_of_reviews = soup.find('span', class_='sc-1svr2te-7 jMATuZ').get_text().replace("[^0-9]", " ")
  except:
    no_of_reviews = ''
  try:
    rankings = soup.find('h2', class_='sc-1svr2te-4 dpMist').get_text()
  except:
    rankings = ''
  try:
    great_for = ','.join([li.get_text() for li in soup.find('div', class_='sc-197szwq-1 bRyoyk').find_all('li')])               # data extractor block
  except:
    great_for = ''
  try:
    not_great_for = ','.join([li.get_text() for li in soup.find_all('div', class_='sc-197szwq-1 bRyoyk')[1].find_all('li')])
  except:
    not_great_for = ''
  try:
    who_lives_here = ','.join([li.get_text() for li in soup.find_all('div', class_='sc-197szwq-1 bRyoyk')[2].find_all('li')])
  except:
    who_lives_here = ''

  australia_timezone = pytz.timezone('Australia/Sydney')
  current_time = datetime.datetime.now(australia_timezone)

  data = {
      'Suburb Name': suburbname,
      'Ratings': ratings,
      'Number of Reviews': no_of_reviews,
      'Rankings': rankings,
      'Great for': great_for,
      'Not great for': not_great_for,
      'Who lives here?': who_lives_here,
      'Time Stamp' : current_time,
      'Url' : suburb,
      'Region' : region
  }

  #file_name = 'HomelyExtract_'+ str(current_time.date()).replace('-', '')+'.csv'
  file_name = 'HomelyExtract1.csv'
  tempdf = pd.DataFrame(data,index=[0])
  save_data_to_disk(tempdf, file_name)
  tempdf = pd.DataFrame()
  driver.delete_all_cookies()
